In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [5]:
# Install only required packages
!pip install langchain langchain-community langchain-groq wikipedia groq -q

# Imports
import os
import wikipedia
from kaggle_secrets import UserSecretsClient
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_core.documents import Document
from langchain.chains.combine_documents import create_stuff_documents_chain

# API key setup
user_secrets = UserSecretsClient()
os.environ["GROQ_API_KEY"] = user_secrets.get_secret("GROQ_API_KEY")

# LangChain LLM setup
llm = ChatGroq(model_name="llama3-70b-8192")

# Define how the LLM should summarize Wikipedia content
prompt = ChatPromptTemplate.from_messages([
    ( "human", """
    Given the following Wikipedia article content, extract:
    - Founder
    - Year Founded
    - Number of Branches/Campuses
    - Number of Employees
    - A 4-line summary

    Content:
    {context}
    """
)])

# Combine Wikipedia document with the prompt
chain = create_stuff_documents_chain(llm=llm, prompt=prompt)

# Institution Wikipedia lookup
institution_name = "T John Institute of Technology"
wiki_content = wikipedia.page(institution_name).content
doc = Document(page_content=wiki_content)
result = chain.invoke({"context": [doc]})
print(result.strip() if isinstance(result, str) else result)

Here are the extracted information and summary:

**Founder:** Thomas P John

**Year Founded:** 2006 (although the T. John Group of Institutions was established in 1993)

**Number of Branches/Campuses:** Not specified, but it's a single college with multiple departments

**Number of Employees:** Not specified

**4-line Summary:**
T. John Institute of Technology is a private engineering college in Bangalore, India. It was established in 2006 and offers undergraduate and postgraduate programmes in six streams of engineering. The college has expanded to include various departments, facilities, and extracurricular activities. It is affiliated to Visvesvaraya Technological University and approved by All India Council for Technical Education.
